In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
import io
from sklearn import svm, base, feature_selection, linear_model
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve,auc,classification_report,f1_score,accuracy_score,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import subprocess
from os import listdir
#to visualize
from sklearn.externals.six import StringIO
# import pydot

D:\Users\baran\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [172]:
# features_random_soc-firm-hi-tech_final
# Train and test paths
train_data_path = "__Random_Features\\features_random_soc-firm-hi-tech_final.csv"
test_data_path = "__Real_Features\\soc-firm-hi-tech_final.csv"

In [173]:
# Read data files
df = pd.read_csv(train_data_path).drop(columns='Unnamed: 0')
model_test = pd.read_csv(test_data_path).drop(columns='Unnamed: 0')

In [4]:
# Prepare data to train. 
# X: Independent values
# y: Target values (Categories)
def prepareData(df, normalize=False):
    X = np.array(df.drop(['Name', 'Categories'], 1))
    if normalize:
        return (X - X.mean()) / (X.max() - X.min()), np.array(df['Categories'])
    return X, np.array(df['Categories'])

In [5]:
def doSVMcvPrediction(Xin,yin,Xtest,ytest,modType):
    #input: training/test data and labels, model type
    #supported models: SVM (l1 and l2), AdaBoost, decision tree, and random forest
    #train with 5-fold cross validation, then test once using test (holdout) data
    #once the best estimator is chosen here, train on the entire dataset (in + test) outside this function
    #output: training accuracy, gereralization accuracy, feature weights/importances, classifier, 
    #  classification report, training f1-score and generalization f1-score
    nfolds = 5
    cv = StratifiedKFold(n_splits=nfolds, shuffle=True)
    #l1 penalty enforces sparsity in weights, only available for linear SVM classifier
    if modType in ('SVM-L2','svm-l2'):
        clasf = svm.LinearSVC(loss='squared_hinge', penalty='l2', tol=.001, dual=False, class_weight='balanced')
        cvclasf = GridSearchCV(clasf, param_grid = {
            'C' : [0.05, 0.1, 0.5, 1, 5, 10, 500, 1000]
            }, verbose=0,refit=True,
            cv=cv,
            # scoring='roc_auc',
            scoring='f1_weighted',
        n_jobs=4)

    elif modType in ('SVM-L1','svm-l1'):
        clasf = svm.LinearSVC(loss='squared_hinge', penalty='l1', tol=.001, dual=False, class_weight='balanced')
        cvclasf = GridSearchCV(clasf, param_grid = {
            'C' : [0.05, 0.1, 0.5, 1, 5, 10, 500, 1000]
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)
    
    #decision tree classifiers
    elif modType in ('ada','adaboost','adaboost-tree'):
        clasf = AdaBoostClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'n_estimators' : [5,10,25,50,100],
            'learning_rate' : [0.1,0.3,0.5]
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)

    elif modType in ('dtree','decision-tree'):
        clasf = DecisionTreeClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'splitter' : ['best'],
            'criterion' : ['entropy','gini'],
            'max_features' : [0.2,'sqrt',1.],
            'max_depth' : [2,4], 
            'class_weight' : ['balanced'], 
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)

    elif modType in ('rf','random-forest'):
        clasf = RandomForestClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'n_estimators' : [5,10,25,50,100],
            'criterion' : ['entropy','gini'],
            'max_features' : [0.2,'sqrt',1.],
            'max_depth' : [2,4], 
            'class_weight' : ['balanced'], 
            }, verbose=0,refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)
    
    elif modType in ('knn', 'kneighbors'):
        clasf = KNeighborsClassifier()
        cvclasf = GridSearchCV(clasf, param_grid = {
            'n_neighbors': [3, 5, 7, 11, 13, 15, 19],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan']
            }, verbose=0, refit=True,
            cv=cv,
            scoring='f1_weighted',
        n_jobs=4)

    cvclasf.fit(Xin,yin)
    bclasf = cvclasf.best_estimator_
    print("%s %d-fold CV params: %s" % (modType,nfolds,cvclasf.best_params_))
    
    if modType in ('ada','adaboost-tree','dtree','decision-tree','rf','random-forest'):
        w = bclasf.feature_importances_
    elif modType in ('SVM-L1','svm-l1','SVM-L2','svm-l2'):
        w = bclasf.coef_
    elif modType in ('knn', 'kneighbors'):
        w = np.zeros(2)
    
    bclasf.fit(Xin,yin)
    y_train_pred = bclasf.predict(Xin)
    acTrain = accuracy_score(yin,y_train_pred)
    f1Train = f1_score(yin,y_train_pred,average="weighted")
    
    y_pred = bclasf.predict(Xtest)
    report = classification_report(ytest, y_pred)
    acGeneral = accuracy_score(ytest, y_pred)
    f1Gen = f1_score(ytest,y_pred,average="weighted")

    return(acTrain,np.squeeze(w),bclasf,report,(acTrain,acGeneral),(f1Train,f1Gen))

In [6]:
def testModel(df_train, df_testing, modelType):
    X, y = prepareData(df_train)
    X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.5, stratify=y)
    
    RGfamilies = ["CL", "CNFG", "GNP", "PA"]
    
    # Preprocess
    scaler1=StandardScaler()
    X_train = scaler1.fit_transform(X_train.astype(np.double))
    X_test = scaler1.transform(X_test.astype(np.double))
    
    
    score,optWeights,clasf,rep,accs,f1s = doSVMcvPrediction(X_train, y_train, X_test, y_test, modelType)
    
    print(rep)
    print()
    
    test_measure = df_testing.drop(columns=['Name'])
    x = np.array(test_measure.values)
    
    
    scaler=StandardScaler()
    X = scaler.fit_transform(X.astype(np.double))
    x = scaler.transform(x.astype(np.double))
    
    
    clasf.fit(X,y)
    novelRG = clasf.predict(x)
    print("\nPrediction is %s\n"%(novelRG))
    
    Fcol = df.drop(['Name', 'Categories'], 1).columns
    
    if modelType in ('SVM-L1','SVM-L2'):
        #TODO: print statements when useSpectral
        print("SVM feature weights:")
        print(np.column_stack((clasf.coef_.T,Fcol)))
        sc = x.dot(clasf.coef_.T) + clasf.intercept_
        print("scores:")
        print(np.vstack((RGfamilies,sc)))
    elif modelType in ('adaboost-tree'):
        print("Adaboost feature weights:")
        print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        # print "prediction probabilities:"
        # print np.vstack((RGfamilies,clasf.predict_proba(x)))
        print("decision function:")
        print(np.vstack((RGfamilies,clasf.decision_function(x))))
        #also the actual tree?
    elif modelType in ('decision-tree'):
        print("Decision tree feature weights:")
        print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        print("prediction probabilities:")
        print(np.vstack((RGfamilies,clasf.predict_proba(x))))
        #also the actual tree
        #Fcol2 = [s.replace('n3','H').replace('n4','F') for s in Fcol]
        #writeTree(clasf,Fcol2,graphFolder[:-1]+'_dTree.pdf')
    elif modelType in ('random-forest'):
        print("Random Forest feature weights:")
        print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        print("prediction probabilities:")
        print(np.vstack((RGfamilies,clasf.predict_proba(x))))
    elif modelType in ('knn'):
        print("K Nearest Neighbors feature weights:")
        #print(np.column_stack((clasf.feature_importances_.T,Fcol)))
        print("prediction probabilities:")
        print(np.vstack((RGfamilies,clasf.predict_proba(x))))

In [ ]:
testModel(df, model_test, "SVM-L1")

In [ ]:
testModel(df, model_test, "SVM-L2")

In [ ]:
testModel(df, model_test, "adaboost-tree")

In [ ]:
testModel(df, model_test, "decision-tree")

In [ ]:
testModel(df, model_test, "random-forest")

In [245]:
testModel(df, model_test, "knn")

knn 5-fold CV params: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
              precision    recall  f1-score   support

          CL       1.00      1.00      1.00        50
        CNFG       1.00      1.00      1.00        50
         GNP       1.00      1.00      1.00        50
          PA       1.00      1.00      1.00        50

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



Prediction is ['CNFG']

K Nearest Neighbors feature weights:
prediction probabilities:
[['CL' 'CNFG' 'GNP' 'PA']
 ['0.0' '1.0' '0.0' '0.0']]
